In [ ]:
#@title Clone github repo and ProteinMPNN
import json, time, os, sys, glob

if not os.path.isdir("Scaffold-Lab"):
  os.system("git clone -q https://github.com/Immortals-33/Scaffold-Lab.git")
sys.path.append('/content/Scaffold-Lab')

In [ ]:
#@title Install Related Libraries
!pip install pydantic==1.10.12
!pip install triton==1.0.0
!pip install rootutils GPUtil biotite

!pip install --upgrade MDAnalysis
!pip install mdtraj tmtools
!pip install pandas
!pip install dm-tree
!pip install hydra-core --upgrade
!pip install esm

!pip install hydra-joblib-launcher --upgrade
!pip install hydra-core --upgrade


In [ ]:
#@title Install PyMol
from IPython.utils import io
import tqdm.notebook
import os
"""The PyMOL installation is done inside two nested context managers. This approach
was inspired by Dr. Christopher Schlicksup's (of the Phenix group at
Lawrence Berkeley National Laboratory) method for installing cctbx
in a Colab Notebook. He presented his work on September 1, 2021 at the IUCr
Crystallographic Computing School. I adapted Chris's approach here. It replaces my first approach
that requires seven steps. My approach was presentated at the SciPy2021 conference
in July 2021 and published in the
[proceedings](http://conference.scipy.org/proceedings/scipy2021/blaine_mooers.html).
The new approach is easier for beginners to use. The old approach is easier to debug
and could be used as a back-up approach.

"""
total = 100
with tqdm.notebook.tqdm(total=total) as pbar:
    with io.capture_output() as captured:

        !pip install -q condacolab
        import condacolab
        condacolab.install()
        pbar.update(10)

        import sys
        sys.path.append('/usr/local/lib/python3.7/site-packages/')
        pbar.update(20)

        # Install PyMOL
        %shell mamba install -c schrodinger pymol-bundle --yes

        pbar.update(90)

        # Mount Google Drive and upload your PyMOL license
        #from google.colab import drive
        #drive.mount("/content/drive")
        #print(os.listdir("/content/drive/MyDrive"))
        #%shell cp ./pymol-edu-license.lic .
        pbar.update(100)


In [ ]:
#@title Config

mode = "motif_scaffolding" #@param ["unconditional", "motif_scaffolding"]
#@markdown - task name : unconditional generation or motif scaffolding
query_pdb_folder = '/content/Scaffold-Lab/demo/motif_scaffolding/2KL8/' #@param {type:"string"}
#@markdown - customised pdb file path, you can direct to your colab path
contig_str = 'none' #@param {type:"string"}
#@markdown - **only motif scaffolding** : information on motifs and scaffolds of each designed protein such as "A1-7/20-20/A28-79"
contig_csv = '/content/Scaffold-Lab/demo/motif_scaffolding/motif_info.csv' #@param {type:"string"}
#@markdown - **only motif scaffolding** : information on motifs and scaffolds of each designed protein, only used in motif_scaffolding problem.This is particularly important when sample length-variable scaffolds for a single case, for different samples correspond to different motif regions.
native_pdb_folder = '/content/Scaffold-Lab/demo/motif_scaffolding/reference_pdbs' #@param {type:"string"}
#@markdown - **only motif scaffolding** : ground truth pdb folder 
metric = "scRMSD_pAE_motifRMSD" #@param ["scRMSD_pLDDT_motifRMSD", "scRMSD_pAE_motifRMSD"]
#@markdown - the in silico computation metric for successful design

In [ ]:
#@title Fixed backbone design and Refolding
if mode == "motif_scaffolding":
  !python /content/Scaffold-Lab/scaffold_lab/motif_scaffolding/motif_refolding_hugginface.py inference.backbone_pdb_dir=f'{query_pdb_folder}' inference.output_dir='./' inference.motif_csv_dir=f'{contig_csv}' inference.input_pdbs_dir=f'{native_pdb_folder}'
elif mode == "unconditional":
  !python /content/Scaffold-Lab/scaffold_lab/unconditional/refolding_huggingface.py inference.backbone_pdb_dir=f'{query_pdb_folder}' inference.output_dir='./'